# Assignment expressions

In [1]:
a = (b:=100) / 2
print(a, b) # => (50.0, 100)

50.0 100


## これが

In [71]:
%%writefile test.txt
Python3.8
が
2019.10.14にリリースされた
よ

Overwriting test.txt


In [72]:
with open("test.txt", "r") as f:
    line = f.readline()
    print(line, end="")
    while line:
        line = f.readline()
        print(line, end="")

Python3.8
が
2019.10.14にリリースされた
よ


## こうかけるように

In [5]:
with open("test.txt", "r") as f:
    while line := f.readline():
        print(line, end="")

Python3.8
が
2019.10.14にリリースされた
よ


## Pythonの父 Guido van RossumのBDFL退任のきっかけ

[Python 3.8 の概要 (その1) - Assignment expressions - atsuoishimoto's diary](https://atsuoishimoto.hatenablog.com/entry/2019/09/03/110508)

> 多くの開発者はこの提案が受け入れられるとは思ってもいなかったし、Pythonの仕様決定権をもつPythonの父 Guido van Rossum 氏もそのつもりはなかったようです。しかし、議論が進むにつれ、なんとGuidoが代入演算子を受け入れる意向を示し始めました。

> 多くのPython開発者はこの提案に反対で、大変な議論が巻き起こりましたが、Guidoを始めとする推進派は丁寧に議論を進め、最終的に PEP 572 -- Assignment Expressions としてPythonに導入されることとなりました。

> この議論で疲れ果てた Guido は、Pythonの最高権力者であるBDFL(Benevolent Dictator For Life:慈悲深き終身独裁官) からの退任を表明することとなります。

#  Positional-only parameters

## 使わない場合

In [8]:
def plus(x, y):
    r = x + y
    return r

In [9]:
plus(1, 2)

3

In [10]:
plus(x=1, y=2)

3

## 使った場合

In [17]:
def plus_with_new(x, y, /):
    r = x + y
    return r

In [18]:
plus_with_new(1, 2)

3

In [19]:
plus_with_new(x=1, y=2)

TypeError: plus_with_new() got some positional-only arguments passed as keyword arguments: 'x, y'

## デフォルト引数は使える

In [21]:
def plus_with_new_and_dflt(x, y, z=None, /):
    r = x + y
    if z:
        r = r * 10
    return r

In [22]:
plus_with_new_and_dflt(1, 2, 3)

30

In [23]:
plus_with_new_and_dflt(1, 2, z=3)

TypeError: plus_with_new_and_dflt() got some positional-only arguments passed as keyword arguments: 'z'

## キーワード引数と組み合わせると

In [25]:
def plus_with_all(a, b, /, c, d, *, e=None, f=None):
    r = a + b + c + d
    if e:
        r += e
    if f:
        r += f
    return r

In [26]:
plus_with_all(1, 2, 3, 4, 5, 6)

TypeError: plus_with_all() takes 4 positional arguments but 6 were given

In [27]:
plus_with_all(1, 2, 3, 4, e=5, f=6)

21

In [28]:
plus_with_all(a=1, b=2, 3, 4, e=5, f=6)

SyntaxError: positional argument follows keyword argument (<ipython-input-28-5492848aea69>, line 1)

In [29]:
plus_with_all(1, 2, c=3, d=4, e=5, f=6)

21

# f-strings “=” 記法をサポート

In [30]:
foo = "Python"
bar = "3.8"

In [31]:
print(f"foo={foo} bar={bar}")

foo=Python bar=3.8


In [33]:
print(f"{foo=} {bar=}")

foo='Python' bar='3.8'


# reversed() が dictに対しても使えるように

In [34]:
my_dict = dict(a=1, b=2)

In [39]:
my_dict

{'a': 1, 'b': 2}

In [42]:
dict(reversed(my_dict.items()))

{'b': 2, 'a': 1}

# math.prod()

In [44]:
import math
math.prod([1,2,3,4])

24

# importlib.metadataモジュール

In [47]:
from importlib.metadata import version, distributions

In [50]:
version('pip')

'19.3.1'

In [52]:
for dist in list(distributions())[:5]:
    print(f"{dist.metadata['Name']}: {dist.version=}")

nbconvert: dist.version='5.6.0'
ipywidgets: dist.version='7.5.1'
pyrsistent: dist.version='0.15.4'
importlib-metadata: dist.version='0.23'
pip: dist.version='19.3.1'


# Did you mean "=="?

さて、質問です。

```
a = 1.0
a is 1.0
```

上記の処理で、a is 1.0 の結果は True となるでしょうか、それとも False となるでしょうか？

True と答えたあなた、不正解です。反省してください。

False と答えたあなた、同じく不正解です。猛省してください。

正解は 「わからない」 です。

In [55]:
a = 1.0
a is 1.0

<>:2: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:2: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-55-b263d0f4c3ec>:2: SyntaxWarning: "is" with a literal. Did you mean "=="?
  a is 1.0


False

In [56]:
...is...

True

# json.tool

In [60]:
from json import tool

In [68]:
%%writefile json.txt
{"city": "tokyo", "temperature": 6}
{"city": "osaka", "temperature": 5}
{"city": "taipei", "temperature": 17}

Overwriting json.txt


In [70]:
!python -m json.tool --json-lines  < json.txt

{
    "city": "tokyo",
    "temperature": 6
}
{
    "city": "osaka",
    "temperature": 5
}
{
    "city": "taipei",
    "temperature": 17
}


# return, yieldでunpacking改善

In [75]:
def old_unpack():
    rest = (4, 5, 6)
    t = 1, 2, 3, *rest
    return t

In [76]:
def new_unpack():
    rest = (4, 5, 6)
    return 1, 2, 3, *rest

In [78]:
old_unpack()

(1, 2, 3, 4, 5, 6)

In [77]:
new_unpack()

(1, 2, 3, 4, 5, 6)

# New syntax warnings

In [79]:
data = [
    (1, 2, 3)  # oops, missing comma!
    (4, 5, 6)
]

<>:2: SyntaxWarning: 'tuple' object is not callable; perhaps you missed a comma?
<>:2: SyntaxWarning: 'tuple' object is not callable; perhaps you missed a comma?
<ipython-input-79-ba54d0bb6938>:2: SyntaxWarning: 'tuple' object is not callable; perhaps you missed a comma?
  (1, 2, 3)  # oops, missing comma!


TypeError: 'tuple' object is not callable